# Opening images in Keras
Todo:
 - Change from Pillow to rasterio to read image
 - 

Sources:
    - https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a
    - https://stackoverflow.com/questions/46493419/use-a-generator-for-keras-model-fit-generator
    - https://stackoverflow.com/questions/50195967/how-can-i-create-a-4d-numpy-array-from-images
    - https://www.google.com/search?client=firefox-b-ab&q=keras+make+image+generator
 

In [12]:
from PIL import Image

import keras
from PIL import Image

import imageio



In [2]:
import rasterio
import numpy as np

with rasterio.open('../Sorted_tiles/testimage.tif', 'r') as ds:
    arr = ds.read()  # read all raster values
    #arr = np.moveaxis(arr, 0, -1) # Moves the channel index to the back.

print(arr.shape) 

import imageio
im = imageio.imread('../Sorted_tiles/testimage.tif')
print(im.shape) 

RasterioIOError: ../Sorted_tiles/testimage.tif: No such file or directory

In [3]:
import glob
import os
list_images = glob.glob('../Sorted_tiles/*.tif')

batch_paths = np.random.choice(a = list_images, 
                                         size = 2)
print(batch_paths)
print(os.path.basename(batch_paths[0]))

['../Sorted_tiles/all_bands_merged_07_55.tif'
 '../Sorted_tiles/all_bands_merged_01_53.tif']
all_bands_merged_07_55.tif


In [4]:
k = np.random.randint(low=1, high=6,  size=30)
print(k)

[5 2 3 3 3 5 1 3 5 3 1 2 4 5 1 3 3 2 2 4 4 5 3 5 5 1 4 1 2 1]


## Read imagelabels

In [105]:
import pandas as pd
labelfile = pd.read_csv('../Qgis/labeled_tiles.csv')
labelfile.head()

,ID,Tile_name,LIVSM1,LIVSM2,mult_bin_labels,1,2,3,4,5,6,7,8,9,10,11,12,99
0,0,all_bands_merged_01_01.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,all_bands_merged_01_02.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,all_bands_merged_01_03.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,all_bands_merged_01_04.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,all_bands_merged_01_05.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1


In [139]:
test = labelfile.loc[0, ['1','2','3','4','5','6','7','8','9','10','11','12','99']].tolist()
print(test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [195]:
test = labelfile[labelfile.Tile_name == 'all_bands_merged_01_01.tif'].loc[0,['1','2','3','4','5','6','7','8','9','10','11','12','99']].tolist()

print(test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [194]:
test = labelfile.loc[labelfile.Tile_name == 'all_bands_merged_01_01.tif', ['1','2','3','4','5','6','7','8','9','10','11','12','99']].tolist()

print(test)

AttributeError: 'DataFrame' object has no attribute 'tolist'

## Homemade image data generator

In [5]:
def flip_vertical(image):
    return np.flipud(image)

def flip_horizontal(image):
    return np.fliplr(image)

def rotate_90(image):
    return np.rot90(image, k=1)

def rotate_180(image):
    return np.rot90(image, k=2)

def rotate_270(image):
    return np.rot90(image, k=3)

def random_image_flip_rotation(image):
    # This function rotates or flips the image randomly using one of five functions. 
    # This function can also do nothing to the image. 1/6 chanche.
    # Returns a image

    randint = np.random.randint(low=1, high=6)
    modified_image = 0
    
    if randint == 1:
        modified_image = flip_vertical(image)
    elif randint == 2:
        modified_image = flip_horizontal(image)

    elif randint == 3:
        modified_image = rotate_90(image)

    elif randint == 4:
        modified_image = rotate_180(image)

    elif randint == 5:
        modified_image = rotate_270(image)

    else:
        modified_image = image
        
    return modified_image

A = np.array([[1, 2, 3], [4, 5, 6], [7,8,9]])
print(A)
print(flip_vertical(A))
print(flip_horizontal(A))
print(rotate_90(A))
print(rotate_180(A))
print(rotate_270(A))
print(random_image_flip_rotation(A))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[7 8 9]
 [4 5 6]
 [1 2 3]]
[[3 2 1]
 [6 5 4]
 [9 8 7]]
[[3 6 9]
 [2 5 8]
 [1 4 7]]
[[9 8 7]
 [6 5 4]
 [3 2 1]]
[[7 4 1]
 [8 5 2]
 [9 6 3]]
[[7 8 9]
 [4 5 6]
 [1 2 3]]


In [237]:
list_images = glob.glob('../Sorted_tiles/*.tif')


def get_input(path):
    
    img = imageio.imread(path)
    
    return(img)


def image_generator(list_images, label_file, batch_size = 1):
    # list_images: list of where the images are placed.
    # label_file: csv file with labels and filenames.
    
    # Check that all images are the same size. Dimentions
    while True:
        batch_input = []
        batch_y = []
        batch_x = 0
        
        
        # Select files (paths/indices) for the batch
        batch_paths = np.random.choice(a = list_images, 
                                     size = batch_size)

        # Read in each input, perform preprocessing and get labels

        for input_path in batch_paths:

          input = get_input(input_path)
          batch_input += [ input ]

          # Set a label by using the tile name
          #label = labelfile[labelfile.Tile_name == os.path.basename(input_path)]['mult_bin_labels']
            
          # Set a label by using the tile name. This line first finds the rowdata for 
          # a spesific tile and gives it out as a dataframe. Then the .iloc function 
          # finds the values of the colums 1 up to 99 and give them out as a list(list in a list). 
          label = labelfile.loc[labelfile.Tile_name == os.path.basename(input_path), ['1','2','3','4','5','6','7','8','9','10','11','12','99']].values

        
        
          batch_y.append(label) # Needs to be int not string, is it working whitout?

        # Return a tuple of (input,output) to feed the network
        batch_x = np.stack(batch_input, axis=0)

        yield(batch_x, batch_y)

In [238]:
test_generated_images = image_generator(list_images, labelfile)
generert = next(test_generated_images)
print(generert)
#print(next(test_generated_images))
print(np.shape(generert[0]))
print(generert[1])

(array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       

In [239]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 50, 474)))
model.add(Activation('sigmoid'))

model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(13, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 48, 48, 32)        136544    
_________________________________________________________________
activation_25 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
global_average_pooling2d_13  (None, 32)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 256)               8448      
_________________________________________________________________
activation_26 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 13)                3341      
Total para

In [245]:
model.fit_generator(
        image_generator(list_images, labelfile, batch_size = 1),
        steps_per_epoch=5,
        epochs=10)

Epoch 1/10
5/5 [==============================] - 1s 111ms/step - loss: 0.0116 - acc: 1.0000
Epoch 2/10
5/5 [==============================] - 1s 102ms/step - loss: 0.0096 - acc: 1.0000
Epoch 3/10
5/5 [==============================] - 1s 101ms/step - loss: 0.0043 - acc: 1.0000
Epoch 4/10
5/5 [==============================] - 0s 98ms/step - loss: 0.4170 - acc: 0.9077
Epoch 5/10
5/5 [==============================] - 0s 99ms/step - loss: 0.0048 - acc: 1.0000
Epoch 6/10
5/5 [==============================] - 1s 101ms/step - loss: 0.0067 - acc: 1.0000
Epoch 7/10
1/5 [=====>........................] - ETA: 0s - loss: 0.0099 - acc: 1.0000

ValueError: Error when checking input: expected conv2d_13_input to have shape (50, 50, 474) but got array with shape (9, 50, 474)

In [14]:
nn = model.predict_generator(
        image_generator(list_images),
        steps=10)

array([[0.50386584],
       [0.50610065],
       [0.5037968 ],
       [0.5038453 ],
       [0.5058131 ],
       [0.5042402 ],
       [0.5049603 ],
       [0.5042402 ],
       [0.5037968 ],
       [0.5030699 ],
       [0.5027083 ],
       [0.5042462 ],
       [0.5037197 ],
       [0.5038573 ],
       [0.504117  ],
       [0.5065854 ],
       [0.5042402 ],
       [0.50286454],
       [0.503915  ],
       [0.5030699 ],
       [0.50330627],
       [0.5042402 ],
       [0.5036756 ],
       [0.503312  ],
       [0.5045645 ],
       [0.50428814],
       [0.5038453 ],
       [0.5058131 ],
       [0.5035124 ],
       [0.50544775]], dtype=float32)

In [19]:
#img = Image.open('../Sorted_tiles/kkkk.tiff')

from osgeo import gdal
import numpy as np
ds = gdal.Open('../Sorted_tiles/testimage.tif')
myarray = np.array(ds.GetRasterBand(1).ReadAsArray())

()


In [ ]:
# Finne ut korleis eg får lest det slik at matrisa blir 4 dimensjonal.
# Check that all images are the same size. Dimentions